<a href="https://colab.research.google.com/github/BrunoReis136/tensorflow/blob/main/TensorFlowTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> TTS -  DOWNLOAD DO DATASET LJSPEECH </h1>

In [ ]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar xvjf LJSpeech-1.1.tar.bz2
#download LJSpeech-1.1  dataset de TTS

<h1>TTS - Funções utilizadas</h1>

In [ ]:
import tensorflow as tf
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast

#--------------------------Áudio para spectrograma
def load_mel(file_path):
  y, sr = librosa.load(file_path, sr=22050)
  mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=256, n_mels=80)
  mel_db = librosa.power_to_db(mel, ref=np.max)
  return mel_db

#--------------------------Texto para token
def tokenize_text(text):
    return [symbol_to_id[c] for c in text if c in symbol_to_id]

#--------------------------Texto para spectrograma
def text_to_mel(text, model, max_len=200):
    # Tokenizar e fazer padding
    tokens = tokenize_text(text)
    token_input = pad_sequences([tokens], maxlen=max_len, padding='post')

    # Prever mel spectrogram
    pred_mel = model.predict(token_input)

    # Remover batch dimension
    mel = pred_mel[0].T  # Transpor para shape (n_mels, time)
    return mel

#--------------------------Spectrograma para audio
def mel_to_audio(mel_spec, sr=22050):
    # Desfaz o dB
    mel_spec = librosa.db_to_power(mel_spec)

    # Reconstrução com Griffin-Lim
    audio = librosa.feature.inverse.mel_to_audio(
        mel_spec, sr=sr, n_fft=1024, hop_length=256, n_iter=60
    )
    return audio

#---------------------------Formação do arquivo WAV
def generate_audio_from_text(text, model, output_path='output.wav'):
    mel = text_to_mel(text, model)
    audio = mel_to_audio(mel)
    sf.write(output_path, audio, samplerate=22050)
    print(f"Áudio salvo em: {output_path}")
    return audio

<h2>TTS - PREPARAÇÃO DOS DADOS </h2>



In [ ]:
import tensorflow as tf
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast

#==============Dataset LJSpeech em dataframe Pandas=================

df = pd.read_csv('LJSpeech-1.1/metadata.csv',
                       sep="|",
                       header=None,
                       names=['file_id', 'text', 'normalized_text'])

audios_path = []

for x in df['file_id']:
  audios_path.append(f'LJSpeech-1.1/wavs/{x}.wav')

df['audio_path'] = audios_path


#===============Função para texto em spectrogram==================
symbols = [
    '_',  # padding
    ' ',  # espaço
    '!', "'",'"', '(', ')', ',', '-', '.', ':', ';', '?',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
    'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'
]
#Variáveis necessárias:
symbol_to_id = {s: i for i, s in enumerate(symbols)}
id_to_symbol = {i: s for i, s in enumerate(symbols)}
vocab_size = len(symbols)

#===================Normalizar dados do dataframe(texto>spectro / áudio>spectro)==============

df['mel_spec'] = df['audio_path'].apply(load_mel)
df['tokenized_text'] = df['normalized_text'].apply(lambda x:tokenize_text(str(x)))

#Padding dos dados em variáveis de lista
x_text = pad_sequences(df['tokenized_text'].tolist(), padding='post')

max_len = max([mel.shape[1] for mel in df['mel_spec']])
y_mel = np.array([
    np.pad(mel, ((0, 0), (0, max_len - mel.shape[1])), mode='constant', constant_values=-80.0).T
    for mel in df['mel_spec']
])

#=================Salvar os dados em disco (poupar RAM para o treinamento)======================
np.save('x_text.npy', x_text)
np.save('y_mel.npy', y_mel)
df.to_pickle('df.pkl')

<h1> TSS - CONSTRUÇÃO MODELO E TREINAMENTO

In [ ]:
import tensorflow as tf
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast
#====================Recuperar os dados de treinamento=====================
x_text = np.load('x_text.npy')
y_mel = np.load('y_mel.npy')
df = pd.read_pickle('df.pkl')

symbols = [
    '_',  # padding
    ' ',  # espaço
    '!', "'",'"', '(', ')', ',', '-', '.', ':', ';', '?',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
    'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'
]

vocab_size = len(symbols)

#====================Construção do modelo==================================
embedding_dim = 256
encoder_units = 512
decoder_units = 1024
mel_dim = df['mel_spec'][0].shape[0]

def build_tts_encoder_decoder():
    #Entrada, embedding
    text_input = layers.Input(shape=(None,), dtype='int32', name='text_input')
    embed = layers.Embedding(input_dim = vocab_size, output_dim=embedding_dim)(text_input)

    #Encoder bidirecional com memória reforçada(LSTM)
    encoder_outputs = layers.Bidirectional(layers.LSTM(encoder_units, return_sequences=True))(embed)

    #Attention(reforço do encoder em si mesmo)
    attention = layers.Attention()([encoder_outputs, encoder_outputs])

    #Decoder para gerar spectrogramas memória reforçada (LSTM)
    decoder_lstm = layers.LSTM(decoder_units, return_sequences=True)(attention)
    mel_output = layers.TimeDistributed(layers.Dense(mel_dim))(decoder_lstm)

    #Montagem do modelo final para retornar
    model = models.Model(inputs=text_input, outputs=mel_output)
    return model

#===========================Aplicação e treinamento do modelo=================================
model = build_tts_encoder_decoder()
model.compile(optimizer='adam', loss='mse')
model.fit(x_text, y_mel, batch_size=32, epochs=10, validation_split=0.1)

#===========================Salvando o modelo=============================
model.save('tts_model.h5')

<h1> TTS - TESTE DE UTILIZAÇÃO DO MODELO </h1>

In [ ]:
from IPython.display import Audio

text = input("Digite o texto: ")
generate_audio_from_text(text, model, output_path='audio_gerado.wav')
Audio("audio_gerado.wav")

<h3> Opcionais </h3>

In [ ]:
#=============Dataframe para excel (opcional para inspeção)===============
#df.to_excel('LJSpeech.xlsx', index=False)

#==============Trecho opcional para executar áudio e exibir spectrogram============

mel_db = load_mel(audio_path)

# AudioDisplay and Plot the Mel spectrogram
n = 10
file_name = df.iloc[n]['file_id']
print(df.iloc[n]['text'])

audio_path = f'LJSpeech-1.1/wavs/{file_name}.wav'

display(Audio(filename=audio_path))

plt.figure(figsize=(10, 4))

librosa.display.specshow(mel_db.T, sr=22050, hop_length=256, x_axis='time', y_axis='mel')

plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram (dB)')
plt.tight_layout()
plt.show()



<h1>NOVA ABORDAGEM</h1>

<h2>CONVERTER ARQUIVOS DE ÁUDIO PARA PADRÕES RATE/CHANNEL/WIDTH</h2>

In [ ]:
!pip install pydub

In [3]:
from pydub import AudioSegment
import os
from tqdm import tqdm

input_dir = "LJSpeech-1.1/wavs"
output_dir = "LJSpeech-1.1/wavs16k"

for filename in tqdm(os.listdir(input_dir)):
  if filename.endswith(".wav"):
    filepath = os.path.join(input_dir, filename)
    audio = AudioSegment.from_wav(filepath)

    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)

    output_path = os.path.join(output_dir, filename)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    audio.export(output_path, format="wav")

100%|██████████| 13100/13100 [00:48<00:00, 272.00it/s]


<h2>Testando o formato dos áudios convertidos</h2>

In [ ]:
import wave

with wave.open("LJSpeech-1.1/wavs16k/LJ001-0001.wav", "rb") as wav_file:
    print("Frequência:", wav_file.getframerate())
    print("Canais:", wav_file.getnchannels())
    print("Sample width:", wav_file.getsampwidth())  # Deve ser 2 (16-bit)

<h2>Dataframe dos metadatas e coluna com caminho dos arquivos</h2>

In [ ]:
import pandas as pd

metadata = pd.read_csv('LJSpeech-1.1/metadata.csv',sep="|", header=None)
metadata.columns = ['file_id', 'text', 'normalized_text']
metadata['wav_path'] = metadata['file_id'].apply(lambda x:f'LJSpeech-1.1/wav16k/{x}.wav')

<h2>Extrair Mel com Librosa: FUNÇÃO</h2>

In [5]:
import os
import librosa
import numpy as np
from tqdm import tqdm

def convert_wav_to_mel(input_dir,
                       output_dir,
                       sr=16000,
                       n_fft=1024,
                       hop_length=256,
                       n_mels=80,
                       power=1.0,
                       to_db=True,
                       verbose=True):

    os.makedirs(output_dir, exist_ok=True)
    file_list = [f for f in os.listdir(input_dir) if f.endswith(".wav")]
    iterator = tqdm(file_list, desc="Convertendo WAV → Mel") if verbose else file_list

    for filename in iterator:
        filepath = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace(".wav", ".npy"))

        y, _ = librosa.load(filepath, sr=sr)

        mel = librosa.feature.melspectrogram(
            y=y,
            sr=sr,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels,
            power=power
        )

        if to_db:
            mel = librosa.power_to_db(mel, ref=np.max)

        np.save(output_path, mel)

<h2>Extrair Mel com Librosa: EXECUÇÃO</h2>

In [6]:
convert_wav_to_mel(
    input_dir="LJSpeech-1.1/wavs16k",
    output_dir="LJSpeech-1.1/mels"
)

Convertendo WAV → Mel: 100%|██████████| 13100/13100 [02:33<00:00, 85.28it/s] 


<h2>Visualização do spectrograma (somente por diversão)</h2>

In [ ]:
import matplotlib.pyplot as plt
import librosa.display

mel = np.load("LJSpeech-1.1/mels/LJ001-0001.npy")
plt.figure(figsize=(10, 4))
librosa.display.specshow(mel, sr=16000, hop_length=256, x_axis="time", y_axis="mel")
plt.colorbar(format="%+2.0f dB")
plt.title("Mel Spectrogram")
plt.tight_layout()
plt.show()

<h2>Dados tabulares para Dataframe</h2>

In [31]:
import pandas as pd

df = pd.read_csv('LJSpeech-1.1/metadata.csv',
                       sep="|",
                       header=None,
                       names=['file_id', 'text', 'normalized_text'])

df['normalized_text'].fillna(df['text'],inplace=True)
df['normalized_text'].dropna(inplace=True)

/tmp/ipython-input-31-2599533971.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['normalized_text'].fillna(df['text'],inplace=True)


<h2>Tokenizar textos: Função</h2>

In [33]:
def build_vocab(texts):
  chars = sorted(set(''.join(texts)))
  chars2idx = {c: i+1 for i, c in enumerate(set(chars))}
  chars2idx['<pad>'] = 0
  idx2chars = {i: c for c, i in chars2idx.items()}
  return chars2idx, idx2chars

In [32]:
texts = df['normalized_text'].tolist()



{'C': 1, 'L': 2, 'P': 3, 'd': 4, 'A': 5, 'K': 6, 't': 7, 'z': 8, '“': 9, 'V': 10, 'q': 11, 'E': 12, 'U': 13, 'c': 14, "'": 15, ',': 16, '(': 17, 'O': 18, 'S': 19, '[': 20, ']': 21, ')': 22, 'Q': 23, 'N': 24, 'D': 25, 'W': 26, 'b': 27, 'F': 28, 'g': 29, 'r': 30, 'f': 31, 'x': 32, 'n': 33, 'â': 34, 'ü': 35, '!': 36, 'j': 37, 'M': 38, 'h': 39, 'T': 40, 'e': 41, 'k': 42, 'l': 43, '.': 44, 'a': 45, 'H': 46, 'B': 47, ':': 48, 'R': 49, 'o': 50, 'é': 51, '|': 52, 'è': 53, 's': 54, 'i': 55, '-': 56, 'y': 57, 'u': 58, 'w': 59, '’': 60, '”': 61, 'I': 62, 'X': 63, 'à': 64, ';': 65, 'p': 66, '"': 67, 'ê': 68, ' ': 69, 'G': 70, 'Z': 71, '?': 72, 'm': 73, 'J': 74, 'Y': 75, 'v': 76, '<pad>': 0}
